# Wireless Project

Practical Lab:

>       Robin Gerzaguet
> 
>   ENSSAT - Université de Rennes 1
> 
>   robin.gerzaguet@enssat.fr
> 
>   6 Rue de Kerampont - CS 80518 22305 LANNION - France

## Chapter 2 

***Wireless Lab***

The project will be done under Julia. Most of the functions will have to be coded and some functions will be provided. The transmitter and receiver description are described in the previous chapter. When necessary, the reader should have an deep look to the appropriate parts.

We emulate in this section a small network where the base station provided a broadband service to several users. Each group of student is associated to a device (i.e a mobile terminal) and has a unique identifier. This identifier will be used to extract the dedicated payload content from the multiplexed stream. The ultimate purpose is to decode the sent text and discover the secret key that has been hide for each user in the frame. In the proposed lab, we will focus on the L2 part: we will suppose that we will have a time frequency grid from the low layer (L1) and that this matrix has been obtained from an over the air transmission. The purpose is to recover the payload associated to your group identifier and discover the secret key hidden in the payload content.
    
To do so, we will have to decode all the channels that are multiplexed in the received LTE frame. Each channel has a specific information that will be used to sequentially obtained all the necessary flags and parameters for the final payload decoding.We will first have to built the necessary functions to recover all the parameters, and then apply the successive decoding functions.

$\framebox $

The LTE channels are not static, their size and their parameters evolve by time (associated to network load and link quality). So your code should be !
- Clean your code and use functions when needed
- Always use variables and parameters: nothing should be hard-coded
- Use structures or classes to have a clean nested code, especially for function calls.

In [3]:
include("data/julia/tfMatrix.jl");

In [15]:
tf = vec(tfMatrix);

In [16]:
using Plots

In [21]:
tf[1:10]; @show length(tf)

length(tf) = 14336


14336